In [1]:
import pandas as pd

In [7]:
path_death = 'causes_of_death.csv'

df_death = pd.read_csv(path_death)
df_death.head(2)

,Year,ZIP Code,Causes of Death,Count,Location
0,1999,90002,SUI,1,"(33.94969, -118.246213)"
1,1999,90005,HOM,1,"(34.058508, -118.301197)"
